# Part 1 getting the City of Toronto data from Wikipedia

Install the beautiful soup module

In [89]:
!pip install bs4

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


Handle the imports

In [90]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Build the URL

In [91]:
# create the Wikipedia URL
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Get the web page contents

In [96]:
wikidata = requests.get(url).text

Create a Beautiful Soup object containing the wikipedia data

In [98]:
soup = BeautifulSoup(wikidata,"html5lib")


create a list of the html tables in the BeautifulSoup object

In [99]:
table_list = soup.find_all('table')

table_index = -1
for index,table in enumerate(table_list):
    if ("Toronto" in str(table)):
        table_index = index

if (table_index == -1):
    print("Table not found")
else:
    print("The table index we require is ", table_index)

The table index we require is  0


create a dictionary to hold the burough and neighborhood data from the table identified above 

In [100]:
toronto_neighborhoods=[]        # empty dictionary
table=table_list[table_index]

Data Cleansing:Iterate through the table. Ignore cells where the row contains 'Not assigned'

In [101]:

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3] # postal code contains 3 characters
        cell['Borough'] = (row.span.text).split('(')[0]    # get the borough
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        toronto_neighborhoods.append(cell)



Create a dataframe containing the Toronto Neighborhood data scraped from the Wikipedia table
Further cleanse the borough column 

In [102]:
# print(table_contents)
df=pd.DataFrame(toronto_neighborhoods)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest',
                                             'East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


Use the .shape method to print the number of rows in the dataframe

In [103]:
df.shape

(103, 3)